In [ ]:
"""
#------------ Get OHLC top 15 cryto - exchange Kraken 
Cryto
bitcoin (btc), ethereum (eth), litecoin (ltc), polkadot (dot), monero (xmr), dogecoin (xdg), stellar lumens (xlm),
ripple (xrp), zcash (zec), nano (nano), tron (trx), bitcoin cash (bch), tezos (xtz), cardano (ada), orchid (oxt)

  
#-------------------------------- Kraken API 
Get recent trades
URL: https://api.kraken.com/0/public/Trades

Input:
    pair = asset pair to get trade data for
    since = return trade data since given id (optional.  exclusive)
    Result: array of pair name and recent trade data

<pair_name> = pair name
    array of array entries(<price>, <volume>, <time>, <buy/sell>, <market/limit>, <miscellaneous>)
last = id to be used as since when polling for new trade data

The Trades endpoint takes an optional parameter named since, which specifies the starting date/time of the data. 
The since value is a UNIX timestamp at nanosecond resolution (a standard UNIX timestamp in seconds with 
9 additional digits).

e.g., a call to the Trades endpoint such as 
https://api.kraken.com/0/public/Trades?pair=xbtusd&since=1559347200000000000 
would return the historical time and sales for XBT/USD from the 1st of June 2019 at 00:00:00 UTC:
#------------------------------------------------------------------------

#---------------- Execution 
1 month 950k ticks = 30min    ---  2019 jan-dec = len (l) -> 6.4M records ~10 to 12h
"""

In [94]:
# imports 
import pandas as pd 
import numpy as np
import datetime
import matplotlib.pyplot as plt
import time, json, requests, sys
from time import time, ctime
import time
from datetime import datetime
from requests.exceptions import HTTPError

In [99]:
def Get_OHLC_Kraken(pair, interval='1440', since=''):
    response_json = {}
    url = 'https://api.kraken.com/0/public/OHLC'  #only last 720 datapoints 
    #pair = cryto
    #interval = '1440'                           # 1440 = 1 day 
    #since = ''                                  # return las 720 datapoints
           
    for i in range(3):
        try:
            response_kraken = requests.post(url, 
            params=
                    {'pair':pair,
                    'interval':interval,     
                    'since':since}, 
                    headers={"content-type":"application/json"})
                
            # If the response was successful, no Exception will be raised
            response_kraken.raise_for_status()
        except HTTPError as http_err:
            print(f'HTTP error occurred: {http_err}')  
        except Exception as err:
            print(f'Other error occurred: {err}')  
        else:
             print('Success, URL found!')
        break
        print('--------------------  try #');print(i)
        time.sleep(5)

    if i == 2: sys.exit('URL error GGC')     

    #print(response_kraken.json())
    #g = input("control 2 : "); print (g)
    return response_kraken.json()


In [109]:
def Process_Kraken_Jason(kraken_json, pair):
    i = 0
    l_price = []
    #pair = 'XXBTZUSD'
    
    while True:
        try:
            new_row = [ctime(kraken_json['result'][pair][i][0]),    # Date 
                       kraken_json['result'][pair][i][4],           # Close
                       kraken_json['result'][pair][i][6]            # Volume 
                       ] 
            #print (new_row)
            l_price += [new_row]
            i += 1           

        except Exception as err:
            print(f'error Process_Kraken_Jason: {err}')  
            break 
        
    return l_price



In [120]:
# ' tranform prices list into a DataFrame '
def l_price_into_df_price(l_price, pair):
    l_labels = ['Date', 'Price_'+pair,'Vol_'+pair]
    df_price = pd.DataFrame.from_records(l_price, columns=l_labels)   # l --> list, l_labels --> column names 
    df_price['Date'] = pd.to_datetime(df_price['Date'])
    df_price.set_index('Date', inplace=True) 

    return(df_price)

In [161]:
    # concatenate dataframe with the global dataframe 
    def Concact_Prices(df_price, ft, df_all_prices):
        if ft == True:
          df_all_prices = pd.concat([df_price], axis="columns", join="outer")
          ft = False
        else:
         df_all_prices = pd.concat([df_all_prices, df_price], axis="columns", join="outer")

        return df_all_prices, ft

In [67]:
kraken_json = Get_OHLC_Kraken()
#kraken_json

Success, URL found!


In [77]:
l_price = Process_Kraken_Jason(kraken_json)
#l_price

error Process_Kraken_Jason: list index out of range


In [90]:
df_price = l_price_into_df_price(l_price)
df_price.head()

,Price,Volume
Date,,
2019-01-17 18:00:00,3612.2,1473.06898369
2019-01-18 18:00:00,3683.0,1871.37313760
2019-01-19 18:00:00,3530.0,2048.98164693
2019-01-20 18:00:00,3533.4,1547.13618753
2019-01-21 18:00:00,3575.8,1985.34497321


In [163]:
#Top Cryto
#bitcoin (btc), ethereum (eth), litecoin (ltc), polkadot (dot), monero (xmr), dogecoin (xdg), stellar lumens (xlm),
#ripple (xrp), zcash (zec), nano (nano), tron (trx), bitcoin cash (bch), tezos (xtz), cardano (ada), orchid (oxt)
cryto_list = ['XXBTZUSD', 'XETHZUSD']
interval = '1440'
since = ''
ft = True 

#crypto_list = ['XXBTZUSD', 'eth', 'ltc', 'dot', 'xmr', 'xdg', 'xlm', 'xrp', 'zec', 'nano', 'trx', 'bch', 'xtz', 'ada', 'oxt']
#crypto_list

for pair in (cryto_list):
    kraken_json = Get_OHLC_Kraken(pair, interval, since)
    #print('\n\n')
    #print(kraken_json)

    l_price = Process_Kraken_Jason(kraken_json, pair)
    #print('\n\n')
    #print(l_price)

    df_price = l_price_into_df_price(l_price, pair)
    #print(df_price.head())

    df_all_prices, ft = Concact_Prices(df_price, ft, df_all_prices)
    #print(df_all_prices.head())



Success, URL found!
error Process_Kraken_Jason: list index out of range
                    Price_XXBTZUSD   Vol_XXBTZUSD
Date                                             
2019-01-17 18:00:00         3612.2  1473.06898369
2019-01-18 18:00:00         3683.0  1871.37313760
2019-01-19 18:00:00         3530.0  2048.98164693
2019-01-20 18:00:00         3533.4  1547.13618753
2019-01-21 18:00:00         3575.8  1985.34497321
Success, URL found!
error Process_Kraken_Jason: list index out of range
                    Price_XXBTZUSD   Vol_XXBTZUSD Price_XETHZUSD  \
Date                                                               
2019-01-17 18:00:00         3612.2  1473.06898369         119.78   
2019-01-18 18:00:00         3683.0  1871.37313760         123.30   
2019-01-19 18:00:00         3530.0  2048.98164693         117.21   
2019-01-20 18:00:00         3533.4  1547.13618753         115.68   
2019-01-21 18:00:00         3575.8  1985.34497321         117.88   

                       Vol_XE

In [164]:
df_all_prices.head()

,Price_XXBTZUSD,Vol_XXBTZUSD,Price_XETHZUSD,Vol_XETHZUSD
Date,,,,
2019-01-17 18:00:00,3612.2,1473.06898369,119.78,32171.65643950
2019-01-18 18:00:00,3683.0,1871.37313760,123.30,40364.40808390
2019-01-19 18:00:00,3530.0,2048.98164693,117.21,46466.49675135
2019-01-20 18:00:00,3533.4,1547.13618753,115.68,37208.70357075
2019-01-21 18:00:00,3575.8,1985.34497321,117.88,49974.21796577


In [ ]:
def Get_OHLC_Kraken():
    response_json = {}
    url = 'https://api.kraken.com/0/public/OHLC'  #only last 720 datapoints 
    pair = 'XXBTZUSD'
    interval = '1440'                           # 1440 = 1 day 
    since = ''                                  # return las 720 datapoints
           
    for i in range(3):
        try:
            response_kraken = requests.post(url, 
            params=
                    {'pair':pair,
                    'interval':interval,     
                    'since':since}, 
                    headers={"content-type":"application/json"})
                
            # If the response was successful, no Exception will be raised
            response_kraken.raise_for_status()
        except HTTPError as http_err:
            print(f'HTTP error occurred: {http_err}')  
        except Exception as err:
            print(f'Other error occurred: {err}')  
        else:
             print('Success, URL found!')
        break
        print('--------------------  try #');print(i)
        time.sleep(5)

    if i == 2: sys.exit('URL error GGC')     

    #print(response_kraken.json())
    #g = input("control 2 : "); print (g)
    return response_kraken.json()


In [10]:
# Main Variables 

#url kraken 
url = 'https://api.kraken.com/0/public/Trades' 

# df to get OHLC
df = pd.DataFrame(columns=['Date', 'Price','Volume', 'BS'])

# list rolling ticks from kraken (much faster than a df)
l = []
l_labels = ['Date', 'Price','Volume', 'BS']

# CSV with OHLC
path_csv = '../Data/'
name_csv = 'ohlc.csv'
OHLC_csv = path_csv + name_csv

# CSV with ticks
t_path_csv = '../Data/'
t_name_csv = 'ticks.csv'
ticks_csv = t_path_csv + t_name_csv
ticks_csv

'../Data/ticks.csv'

In [12]:
# Define & Manage Dates 

## 1st since for kraken 
from datetime import datetime
since = datetime(2019, 1, 1, 00, 00, 00000)  
since = datetime.timestamp(since)
print("timestamp =", since)
since = str(since)
since = since[0:10] + '0'* 9
print(since)

# Dates while loop   
now = datetime(2019, 1, 1, 00, 00, 00000)
print(now)
now = int(datetime.timestamp(now))
print(now)

end = datetime(2019, 12, 30, 23, 30, 00000)
print(end)
end = int(datetime.timestamp(end))
print(end)
#g = input("dates range : ") 

# time control scrip execution
time_start = ctime(int(time.time()))
time_end = ctime(int(time.time()))
print(time_start)
print(time_end)

timestamp = 1546322400.0
1546322400000000000
2019-01-01 00:00:00
1546322400
2019-12-30 23:30:00
1577770200
Tue Jan  5 21:12:08 2021
Tue Jan  5 21:12:08 2021


In [19]:
from datetime import datetime

timestamp = 1547769600
dt_object = datetime.fromtimestamp(timestamp)

print("dt_object =", dt_object)
print("type(dt_object) =", type(dt_object))


dt_object = 2019-01-17 18:00:00
type(dt_object) = <class 'datetime.datetime'>


In [ ]:
while now <= end:
    
    print('time --> {}    date --> {}'.format(now, ctime(now)))
    
    time.sleep(1.75)

    for i in range(3):
        try:
            response_kraken = requests.post(url, 
                                            params=
                                            {'pair':'XBTUSD',
                                             'since':since},
                                             headers={"content-type":"application/json"}
                                             )
            # If the response was successful, no Exception will be raised
            response_kraken.raise_for_status()
        except HTTPError as http_err:
            print(f'HTTP error occurred: {http_err}')  # Python 3.6
        except Exception as err:
            print(f'Other error occurred: {err}')  # Python 3.6
        else:
            print('Success, URL found!')
            break
        print('--------------------  try #');print(i)
        time.sleep(8)

    if i == 2: sys.exit('URL error GGC')     

    response_json = response_kraken.json()  
    #print(response_json)                    

    since = response_json['result']['last']

    i = 0
    while True:

        try:
            new_row = [ctime(response_json['result']['XXBTZUSD'][i][2]),
                       response_json['result']['XXBTZUSD'][i][0],
                       response_json['result']['XXBTZUSD'][i][1],
                       response_json['result']['XXBTZUSD'][i][3]
                       ] 
            print (new_row)
            l += [new_row]
            i += 1
            now = int((response_json['result']['XXBTZUSD'][i][2]))
            

        except Exception as err:
            print(f'end while: {err}')  # Python 3.6
            break 



In [ ]:
# ' tranform list into a DataFrame df and work out  OHLC .... '
print (' tranform list into a DataFrame df and work out  OHLC .... ')
df = pd.DataFrame.from_records(l, columns=l_labels)   # l --> list, l_labels --> column names 
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True) 

data_OHLC = pd.to_numeric(df['Price']).resample('30Min').ohlc()



In [ ]:
# export OHLC to csv
data_OHLC.to_csv(OHLC_csv)

# export all ticks to csv
l[1:10]
l[-10:-1]
df.to_csv(ticks_csv)



In [ ]:
# this whole script --> how long does it last?
print(); print()
print (' TimeStart --->  {}'.format(time_start))
time_end = ctime(int(time.time()))
print (' TimeEnd --->    {}'.format(time_end))
print()
print (' len l --->      {}'.format(len(l)))


g = input("end of script : ") 